##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Respuesta a preguntas BERT vía Model Maker de TensorFlow Lite

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/question_answer"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/lite/models/modify/model_maker/question_answer.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/lite/models/modify/model_maker/question_answer.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/lite/models/modify/model_maker/question_answer.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

La librería [Model Maker de TensorFlow Lite](https://www.tensorflow.org/lite/models/modify/model_maker) simplifica el proceso de adaptación y conversión de un modelo TensorFlow a unos datos de entrada concretos cuando se implementa este modelo para aplicaciones de ML en dispositivos.

Este bloc muestra un ejemplo de principio a fin que utiliza la librería Model Maker para ilustrar la adaptación y conversión de un modelo de pregunta-respuesta de uso común para una tarea de pregunta-respuesta.

# Introducción a la tarea de Respuesta a preguntas BERT

La tarea admitida en esta librería es la de pregunta-respuesta extractiva, lo que significa que dado un pasaje y una pregunta, la respuesta es el alcance en el pasaje. La imagen siguiente muestra un ejemplo de pregunta-respuesta.

<p align="center"><img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_squad_showcase.png" width="500"></p>

<p align="center">
    <em>Las respuestas son bloques en el pasaje (crédito de la imagen: <a href="https://rajpurkar.github.io/mlx/qa-and-squad/">SQuAD blog</a>)</em>
</p>

En cuanto al modelo de tarea de pregunta-respuesta, las entradas deben ser el pasaje y el par de preguntas ya preprocesados, las salidas deben ser los logits de inicio y los logits finales de cada token del pasaje. El tamaño de la entrada podría fijarse y ajustarse en función de la longitud del pasaje y de la pregunta.

## Visión general de principio a fin


El siguiente fragmento de código demuestra cómo entrar en el modelo con unas pocas líneas de código. El proceso general incluye 5 pasos: (1) seleccionar un modelo, (2) cargar datos, (3) reentrenar el modelo, (4) evaluar, y (5) exportarlo al formato TensorFlow Lite.

```python
# Chooses a model specification that represents the model.
spec = model_spec.get('mobilebert_qa')

# Gets the training data and validation data.
train_data = DataLoader.from_squad(train_data_path, spec, is_training=True)
validation_data = DataLoader.from_squad(validation_data_path, spec, is_training=False)

# Fine-tunes the model.
model = question_answer.create(train_data, model_spec=spec)

# Gets the evaluation result.
metric = model.evaluate(validation_data)

# Exports the model to the TensorFlow Lite format with metadata in the export directory.
model.export(export_dir)
```

Las siguientes secciones explican el código con más detalle.

## Requisitos previos

Para ejecutar este ejemplo, instale los paquetes necesarios, incluido el paquete Model Maker del repositorio [GitHub](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q tflite-model-maker-nightly

Importe los paquetes necesarios.

In [ ]:
import numpy as np
import os

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import question_answer
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.question_answer import DataLoader

La "Visión general de principio a fin" muestra un sencillo ejemplo de este tipo. Las secciones siguientes recorren el ejemplo paso a paso para mostrar más detalles.

## Elegir un model_spec que represente un modelo para la respuesta a la pregunta

Cada objeto `model_spec` representa un modelo específico para responder a una pregunta. El Model Maker admite actualmente los modelos MobileBERT y BERT-Base.

Modelo compatible | Nombre de model_spec | Descripción del modelo
--- | --- | ---
[MobileBERT](https://arxiv.org/pdf/2004.02984.pdf) | 'mobilebert_qa' | 4.3 veces más pequeño y 5.5 veces más rápido que la BERT-Base logrando resultados competitivos, adecuado para el escenario en el dispositivo.
[MobileBERT-SQuAD](https://arxiv.org/pdf/2004.02984.pdf) | 'mobilebert_qa_squad' | La misma arquitectura de modelo que el modelo MobileBERT y el modelo inicial ya está reentrenado en [SQuAD1.1](https://rajpurkar.github.io/SQuAD-explorer/).
[BERT-Base](https://arxiv.org/pdf/1810.04805.pdf) | 'bert_qa' | Modelo BERT estándar muy utilizado en tareas de PNL.

En este tutorial, se utiliza [MobileBERT-SQuAD](https://arxiv.org/pdf/2004.02984.pdf) como ejemplo. Dado que el modelo ya está reentrenado en [SQuAD1.1](https://rajpurkar.github.io/SQuAD-explorer/), podría tener una cobertura más rápida para la tarea de respuesta a preguntas.


In [ ]:
spec = model_spec.get('mobilebert_qa_squad')

## Cargar datos de entrada específicos para una aplicación de ML en el dispositivo y preprocesar los datos

El [TriviaQA](https://nlp.cs.washington.edu/triviaqa/) es un conjunto de datos de Comprensión lectora que contiene más de 650K tripletas pregunta-respuesta-evidencia. En este tutorial, usará un subconjunto de este conjunto de datos para aprender a usar la librería Model Maker.

Para cargar los datos, convierta el conjunto de datos TriviaQA al formato [SQuAD1.1](https://rajpurkar.github.io/SQuAD-explorer/) ejecutando el [script Python del convertidor](https://github.com/mandarjoshi90/triviaqa#miscellaneous) con `--sample_size=8000` y un conjunto de datos `web`. Modifique un poco el código de conversión:

- Omitiendo los muestreos en los que no pudo encontrar ninguna respuesta en el documento de contexto;
- Entrar en la respuesta original en el contexto sin mayúsculas ni minúsculas.

Descargue la versión archivada del conjunto de datos ya convertido.

In [ ]:
train_data_path = tf.keras.utils.get_file(
    fname='triviaqa-web-train-8000.json',
    origin='https://storage.googleapis.com/download.tensorflow.org/models/tflite/dataset/triviaqa-web-train-8000.json')
validation_data_path = tf.keras.utils.get_file(
    fname='triviaqa-verified-web-dev.json',
    origin='https://storage.googleapis.com/download.tensorflow.org/models/tflite/dataset/triviaqa-verified-web-dev.json')

También puede entrenar el modelo MobileBERT con su propio conjunto de datos. Si está ejecutando este bloc en Colab, cargue sus datos usando la barra lateral izquierda.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_question_answer.png" width="800" hspace="100" alt="Subir archivo">

Si prefiere no subir sus datos a la nube, también puede ejecutar la librería sin conexión siguiendo la [guía](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

Use el método `DataLoader.from_squad` para cargar y preprocesar los datos del formato [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) de acuerdo con un `model_spec` específico. Puede usar los formatos SQuAD2.0 o SQuAD1.1. Establecer el parámetro `version_2_with_negative` como `True` significa que el formato es SQuAD2.0. En caso contrario, el formato es SQuAD1.1. De forma predeterminada, `version_2_with_negative` es `False`.

In [ ]:
train_data = DataLoader.from_squad(train_data_path, spec, is_training=True)
validation_data = DataLoader.from_squad(validation_data_path, spec, is_training=False)

## Personalice el modelo TensorFlow.

Cree un modelo personalizado de pregunta-respuesta basado en los datos cargados. La función `create` comprende los siguientes pasos:

1. Crea el modelo de respuesta a la pregunta según `model_spec`.
2. Entrene el modelo de pregunta-respuesta. Las épocas predeterminadas y el tamaño predeterminado del lote se establecen según dos variables `default_training_epochs` y `default_batch_size` en el objeto `model_spec`.

In [ ]:
model = question_answer.create(train_data, model_spec=spec)

Vea la estructura detallada del modelo.

In [ ]:
model.summary()

## Evalúe el modelo personalizado

Evalúe el modelo en los datos de validación y obtenga un dictado de métricas que incluyan `f1` puntuaciones y `exact match` etc. Tenga en cuenta que las métricas son diferentes para SQuAD1.1 y SQuAD2.0.

In [ ]:
model.evaluate(validation_data)

## Exporte al modelo TensorFlow Lite.

Convierta el modelo entrenado al formato de modelo TensorFlow Lite con [metadatos](https://www.tensorflow.org/lite/models/convert/metadata) para poder usarlo posteriormente en una aplicación de ML en el dispositivo. El archivo de vocabulario está incrustado en los metadatos. El nombre de archivo TFLite predeterminado es `model.tflite`.

En muchas aplicaciones de ML en el dispositivo, el tamaño del modelo es un factor importante. Por lo tanto, se recomienda aplicar la cuantización del modelo para hacerlo más pequeño y, potencialmente, ejecutarlo más rápido. La técnica de cuantización predeterminada tras el entrenamiento es la cuantización de rango dinámico para los modelos BERT y MobileBERT.

In [ ]:
model.export(export_dir='.')

Puede usar el archivo del modelo de TensorFlow Lite en la app de referencia [bert_qa](https://github.com/tensorflow/examples/tree/master/lite/examples/bert_qa/android) usando la [API BertQuestionAnswerer](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_question_answerer) en la [TensorFlow Lite Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview) descargándola de la barra lateral izquierda de Colab.

Los formatos de exportación permitidos pueden ser uno o varios de los siguientes:

- `ExportFormat.TFLITE`
- `ExportFormat.VOCAB`
- `ExportFormat.SAVED_MODEL`

De forma predeterminada, sólo exporta el modelo TensorFlow Lite con metadatos. También puede exportar selectivamente diferentes archivos. Por ejemplo, se puede exportar sólo el archivo de vocabulario de la siguiente manera:

In [ ]:
model.export(export_dir='.', export_format=ExportFormat.VOCAB)

También puede evaluar el modelo tflite con el método `evaluate_tflite`. Se espera que este paso le lleve mucho tiempo.

In [ ]:
model.evaluate_tflite('model.tflite', validation_data)

## Uso avanzado

La función `create` es la parte crítica de esta librería en la que el parámetro `model_spec` define la especificación del modelo. Actualmente se admite la clase `BertQASpec`. Existen 2 modelos: Modelo MobileBERT, Modelo BERT-Base. La función `create` comprende los siguientes pasos:

1. Crea el modelo de respuesta a la pregunta según `model_spec`.
2. Entrene el modelo de pregunta-respuesta.

Esta sección describe varios temas avanzados, como el ajuste del modelo, el ajuste de los hiperparámetros de entrenamiento, etc.

### Ajuste el modelo

Puede ajustar la infraestructura del modelo como los parámetros `seq_len` y `query_len` en la clase `BertQASpec`.

Parámetros ajustables para el modelo:

- `seq_len`: Longitud del pasaje a introducir en el modelo.
- `query_len`: Longitud de la pregunta para introducirla en el modelo.
- `doc_stride`: El intervalo cuando se hace un enfoque de ventana deslizante para tomar trozos de los documentos.
- `initializer_range`: El stdev del truncated_normal_initializer para inicializar todas las matrices de ponderación.
- `trainable`: Booleano, si la capa preentrenada es entrenable.

Parámetros ajustables para la tubería de entrenamiento:

- `model_dir`: La ubicación de los archivos de puntos de verificación del modelo. Si no está activa, se usará el directorio temporal.
- `dropout_rate`: La tasa de abandono.
- `learning_rate`: La tasa de aprendizaje inicial de Adam.
- `predict_batch_size`: Tamaño del lote para la predicción.
- `tpu`: Dirección TPU a la que conectarse. Sólo se utiliza si se usa la TPU.


Por ejemplo, puede entrenar el modelo con una longitud de secuencia mayor. Si cambia el modelo, primero debe construir un nuevo `model_spec`.

In [ ]:
new_spec = model_spec.get('mobilebert_qa')
new_spec.seq_len = 512

Los pasos restantes son los mismos. Tenga en cuenta que debe volver a ejecutar las partes `dataloader` y `create`, ya que las diferentes especificaciones del modelo pueden tener diferentes pasos de preprocesamiento.


### Ajuste los hiperparámetros de entrenamiento

También puede ajustar los hiperparámetros de entrenamiento como `epochs` y `batch_size` para influir en el rendimiento del modelo. Por ejemplo:

- `epochs`: Un mayor número de épocas podría lograr un mejor rendimiento, pero podría conducir a un sobreajuste.
- `batch_size`: número de muestras a usar en un paso de entrenamiento.

Por ejemplo, puede entrenar con más épocas y con un tamaño de lote mayor como:

```python
model = question_answer.create(train_data, model_spec=spec, epochs=5, batch_size=64)
```

### Cambiar la arquitectura del modelo

Puede cambiar el modelo base sobre el que se entrenan sus datos modificando el `model_spec`. Por ejemplo, para cambiar al modelo BERT-Base, ejecute:

```python
spec = model_spec.get('bert_qa')
```

Los pasos restantes son los mismos.

### Personalice la cuantización posterior al entrenamiento en el modelo TensorFlow Lite

[La cuantización posterior al entrenamiento](https://www.tensorflow.org/lite/performance/post_training_quantization) es una técnica de conversión que puede reducir el tamaño del modelo y la latencia de la inferencia, al tiempo que mejora la velocidad de inferencia de la CPU y del acelerador de hardware, con una pequeña degradación de la precisión del modelo. Por ello, se usa mucho para optimizar el modelo.

La librería Model Maker aplica una técnica predeterminada de cuantización postentrenamiento al exportar el modelo. Si desea personalizar la cuantización postentrenamiento, Model Maker también soporta múltiples opciones de cuantización postentrenamiento utilizando [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig). Tomemos como ejemplo la cuantización float16. En primer lugar, defina la configuración de cuantificación.

```python
config = QuantizationConfig.for_float16()
```

A continuación, exportamos el modelo TensorFlow Lite con dicha configuración.

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# Más información

Puede leer nuestro ejemplo de [Preguntas y respuestas de BERT](https://www.tensorflow.org/lite/examples/bert_qa/overview) para saber los detalles técnicos. Para más información, consulte:

- [Guía](https://www.tensorflow.org/lite/models/modify/model_maker) y [Referencia de API](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker) de Model Maker de TensorFlow Lite.
- Librería de tareas: [BertQuestionAnswerer](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_question_answerer) para su implementación.
- Las apps de referencia de principio a fin: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/bert_qa/android) y [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/bert_qa/ios).